In [1]:
# from pymystem3 import Mystem
# m = Mystem()

Выбрала pymorphy, а не mystem, потому что pymorphy позволяет выбирать из нескольких вариантов разбора (если автоматический не подошел). mystem предлагает неверный вариант вариант в 'Даша мыла яблоки'. pymorphy тоже, но можно выбрать нужный.

Сначала импортируем pymorphy и создадим экземпляр класса для анализатора. 

In [2]:
import pymorphy2
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [3]:
corpus = ["Мама мыла раму", "Мама мыла раму", "Даша мыла яблоки", "Даша очень любит маму"]

Напишем функцию для лемматизации элементов списка.

In [4]:
def normalize(text):
    words = text.split()  # Разобьем на слова, потому что pymorphy работает с отдельными словами
    lemmas = list()  # В этот список будем складывать леммы
    for word in words:
        parse = morph.parse(word)  # Парсим каждое слово
        if len(parse) > 2:  # Отловим проблемное слово, где много вариантов разбора, в которых путается pymorphy  
            for p in parse:
                if 'VERB' in p.tag:  # Ищем тот вариант разбора, где глагол 'мыть'
                    lemmas.append(p.normal_form)
        else:
            lemmas.append(parse[0].normal_form)  # Во всех остальных случаях берем первый вариант разбора, он верный
    lem = ' '.join(lemmas)  # По условию задачи объединяем отдельные леммы в список
    return(lem)

Вызовем функцию для каждого элемента корпуса. Получим нужный результат

In [5]:
corpus_lem = list()
for i in corpus:
    corpus_lem.append(normalize(i))
print(corpus_lem)

['мама мыть рама', 'мама мыть рама', 'даша мыть яблоко', 'даша очень любить мама']


Делим полученный лемматизированный корпус на токены и создадим словарь:

In [6]:
corpus_tokenized = [sentence.split() for sentence in corpus_lem]
corpus_tokenized

[['мама', 'мыть', 'рама'],
 ['мама', 'мыть', 'рама'],
 ['даша', 'мыть', 'яблоко'],
 ['даша', 'очень', 'любить', 'мама']]

In [7]:
word_indices = {}

for sentence in corpus_tokenized:
    for word in sentence:
        word_indices[word] = 0

Я костылем его заполнила значениями по порядку, может решение не изящное, но после плясок с пайморфи времени не осталось на продумывание изящности.

In [8]:
v = 0
for k in word_indices.keys():
    word_indices[k] = v
    v += 1
word_indices  

{'мама': 0,
 'мыть': 1,
 'рама': 2,
 'даша': 3,
 'яблоко': 4,
 'очень': 5,
 'любить': 6}

Создадим словарь-гистограмму с частотностью слов.

In [9]:
count_dictionary = {}

for sentence in corpus_tokenized:
    words = set(sentence)  # Убираем дубликаты
    for word in words:
        # Заполняем частотный словарь - количество документов, в которых встретилось слово 
        count_dictionary[word] = count_dictionary.get(word, 0) + 1
count_dictionary

{'рама': 2,
 'мыть': 3,
 'мама': 3,
 'яблоко': 1,
 'даша': 2,
 'очень': 1,
 'любить': 1}

В формуле idf, как помнится мы выяснили, применяется десятичный логарифм, а не натуральный. Вроде бы выбор десятичного или натурального логарифма на конечный результат не влияет, но я оставлю десятичный.

In [10]:
from math import log

In [11]:
idf = {}
for k,v in count_dictionary.items():
    idf[k] = log(len(corpus)/v)  # Формула для вычисления idf, данные берем из частотного словаря 
idf

{'рама': 0.6931471805599453,
 'мыть': 0.28768207245178085,
 'мама': 0.28768207245178085,
 'яблоко': 1.3862943611198906,
 'даша': 0.6931471805599453,
 'очень': 1.3862943611198906,
 'любить': 1.3862943611198906}

Инициализируем векторы для каждого документа в корпусе, они пока заполнены нулевыми значениями.

In [12]:
tf_idf = [[0] * len(idf) for i in range(len(corpus_tokenized))]

In [13]:
from collections import Counter

In [14]:
# "Пронумеруем" каждый список токенов в корпусе, чтобы потом заполнить соответствующие векторы
for n, sentence in enumerate(corpus_tokenized):
    words_counter = Counter(sentence)
    sentence_length = len(sentence)
    # Пройдем по каждому слову и вычислим для него tf-idf
    for word in words_counter:
        word_index = word_indices[word]
        word_tf_idf = (idf[word]/sentence_length)
        print(word, '-->', word_tf_idf) # Посмотрим tf-idf каждого слова
        # Заполним соответствующие ячейки векторов
        tf_idf[n][word_index] = word_tf_idf
tf_idf # Посмотрим на векторы

мама --> 0.09589402415059362
мыть --> 0.09589402415059362
рама --> 0.23104906018664842
мама --> 0.09589402415059362
мыть --> 0.09589402415059362
рама --> 0.23104906018664842
даша --> 0.23104906018664842
мыть --> 0.09589402415059362
яблоко --> 0.46209812037329684
даша --> 0.17328679513998632
очень --> 0.34657359027997264
любить --> 0.34657359027997264
мама --> 0.07192051811294521


[[0.09589402415059362, 0.09589402415059362, 0.23104906018664842, 0, 0, 0, 0],
 [0.09589402415059362, 0.09589402415059362, 0.23104906018664842, 0, 0, 0, 0],
 [0, 0.09589402415059362, 0, 0.23104906018664842, 0.46209812037329684, 0, 0],
 [0.07192051811294521,
  0,
  0,
  0.17328679513998632,
  0,
  0.34657359027997264,
  0.34657359027997264]]

Теперь вычислим косинусное сходство, чтобы понять, какие предлождения ближе друг к другу. 

In [15]:
from scipy.spatial.distance import cosine

In [16]:
result1 = 1 - cosine(tf_idf[0], tf_idf[1])
result2 = 1 - cosine(tf_idf[1], tf_idf[2])
result3 = 1 - cosine(tf_idf[0], tf_idf[2])
result4 = 1 - cosine(tf_idf[0], tf_idf[3])

print('Первое и второе: ', result1)
print('Второе и третье: ', result2)
print('Первое и третье: ', result3)
print('Первое и четвертое: ', result4)

Первое и второе:  1
Второе и третье:  0.06532091836893661
Первое и третье:  0.06532091836893661
Первое и четвертое:  0.04905171616697024


Ближе всего первое и второе предложение.